# Experiments for Paper



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from nn_src.imports import *

/export/home/srasp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
DATA_DIR = '/scratch/srasp/ppnn_data/'
RESULTS_DIR = './tmp'

In [4]:
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

In [5]:
def ensemble_scores(m, n, x_trn, y_trn, x_test, y_test, **kwargs):
    trn_scores, test_scores, preds = [], [], []
    for i in tqdm(range(n)):
        reset_weights(m)
        m.fit(x_trn, y_trn, **kwargs)
        trn_scores.append(m.evaluate(x_trn, y_trn, 4096, verbose=0))
        test_scores.append(m.evaluate(x_test, y_test, 4096, verbose=0))
        preds.append(m.predict(x_test, 4096, verbose=0))
    return trn_scores, test_scores, preds

In [6]:
def save_ensemble(preds, test_set, exp_name, save=True):
    preds = np.array(preds)
    preds[:, :, 1] = np.abs(preds[:, :, 1])   # Make sure std is positive
    mean_preds = np.mean(preds, 0)
    ens_score = crps_normal(mean_preds[:, 0], mean_preds[:, 1], aux_test_set.targets).mean()
    print(f'Ensemble test score = {ens_score}')
    if save:
        results_df = create_results_df(test_set.date_strs, test_set.station_ids, preds[:, 0], preds[:, 1])
        results_df.to_csv(f'{RESULTS_DIR}{fxp_name}.csv')

In [10]:
def get_datasets(pickled_name, train_dates, test_dates=['2016-01-01', '2017-01-01'], aux=False, reload=False):
    pickle_fn = f'{DATA_DIR}pickled/{pickled_name}'
    if not os.path.exists(pickle_fn) or reload:
        var_dict = aux_dict if aux else None
        train_set, test_set = get_train_test_sets(
            DATA_DIR,
            train_dates,
            test_dates,
            aux_dict=var_dict,
        )
        # Save pickled dataset
        with open(pickle_fn, 'wb') as f:
            pickle.dump((train_set, test_set), f)
    else:
        with open(pickle_fn, 'rb') as f:
            train_set, test_set = pickle.load(f)
    return train_set, test_set

## Train 2015

### Neural net with auxiliary data and station embeddings

In [29]:
aux_train_set, aux_test_set = get_datasets('aux_15_16.pkl', ['2015-01-01', '2016-01-01'])

In [15]:
n_features = aux_train_set.features.shape[1]; n_features

40

In [16]:
emb_size = 2
max_id = int(np.max([aux_train_set.cont_ids.max(), aux_test_set.cont_ids.max()]))
max_id

536

In [13]:
emb_model = build_emb_model(n_features, 2, [50], emb_size, max_id, compile=True, lr=0.01)

In [26]:
emb_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 2)         1074        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2)            0           embedding_1[0][0]                
__________________________________________________________________________________________________
concatenat

In [15]:
trn_scores, test_scores, preds = ensemble_scores(
    emb_model, 10,
    [aux_train_set.features, aux_train_set.cont_ids], aux_train_set.targets,
    [aux_test_set.features, aux_test_set.cont_ids], aux_test_set.targets,
    epochs=30, batch_size=1024, verbose=0,
)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [23]:
save_ensemble(preds, aux_test_set, '', False)

Ensemble test score = 0.8250201193851816


## Train 2008-2015

In [11]:
aux_train_set, aux_test_set = get_datasets('aux_08_16.pkl', ['2008-01-01', '2016-01-01'], aux=True)

train set contains 2922 days
test set contains 366 days


In [17]:
emb_model = build_emb_model(n_features, 2, [512], emb_size, max_id, compile=True, lr=0.002)

In [18]:
emb_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 2)         1074        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2)            0           embedding_1[0][0]                
__________________________________________________________________________________________________
concatenat

In [27]:
callbacks = [EarlyStopping(monitor='val_loss',patience=3)]

In [36]:
trn_scores, test_scores, preds = ensemble_scores(
    emb_model, 5,
    [aux_train_set.features, aux_train_set.cont_ids], aux_train_set.targets,
    [aux_test_set.features, aux_test_set.cont_ids], aux_test_set.targets,
    epochs=15, batch_size=4096, verbose=1, callbacks=None, 
)

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

Epoch 1/15
 348160/1456977 [======>.......................] - ETA: 4s - loss: 2.1492

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/export/home/srasp/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/export/home/srasp/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/export/home/srasp/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



1456977/1456977 [==============================] - 6s 4us/step - loss: 1.1987
Epoch 2/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8519
Epoch 3/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8406
Epoch 4/15
1456977/1456977 [==============================] - 5s 3us/step - loss: 0.8330
Epoch 5/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8267
Epoch 6/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8219
Epoch 7/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8182
Epoch 8/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8142
Epoch 9/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8113
Epoch 10/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8071
Epoch 11/15
1456977/1456977 [==============================] - 6s 4us/step - loss: 0.8045
Epoch 12/15
1456977/1456977 [=

In [37]:
test_scores

[0.7845812783046421,
 0.785648775726266,
 0.7820323736911864,
 0.7842717505423521,
 0.779894316266704]

In [38]:
save_ensemble(preds, aux_test_set, '', False)

Ensemble test score = 0.7754476315144285
